# Annotation of Erdman genome
This notebook aims to create a high quality annotation of the Erdman genome. The approach is the following:
1. Extract annotation features from the RATT annotation made based on H37Rv.
2. Extract annotation features made _de novo_ with Bakta.
3. From the Bakta annotations, keep only those non-overlapping with RATT annotation
4. Build genbank record
5. Manually fix CDS transferred with invalid ORFs, add translations and assign final Erdman locus tag.
6. Build feature table
7. Run quality control

In [179]:
from Bio import SeqIO
from io import StringIO
from Bio.SeqFeature import SeqFeature, FeatureLocation
import pandas as pd
import bioframe as bf
from Bio.Seq import Seq

## 1. Extract features from RATT annotation
Made based on H37Rv

In [180]:
def fix_embl_id_line(filename):
    '''
    ratt makes strange ID line in the embl file,
    that biopython doesn't like. This is a quick fix to read it.
    '''
    with open(filename, 'r') as file:
        lines = file.readlines()

    # new_first_line = 'ID                   Erdmann; ; ; ; ; ; 0 BP.'
    new_first_line = 'ID   Erdman; ; circular; genomic DNA; ; ; 0 BP.'
    lines[0] = new_first_line + '\n'
    modified_embl = ''.join(lines)
    
    return StringIO(modified_embl)

In [181]:
ratt_file = '../../Results/8_RATT_annotation/RATT_AssemblyRepetitive/Erdmann.Assembly.Repetitive.contig_1.final.embl'
bakta_file = '../../Results/7_Bakta_annotation/result.gbff'

In [182]:
# write ratt embl output to gb
ratt_gb = '../../Results/8_RATT_annotation/RATT_AssemblyRepetitive/Erdmann.Assembly.Repetitive.contig_1.final.gb'
SeqIO.write(list(SeqIO.parse(fix_embl_id_line(ratt_file), 'embl')), ratt_gb, 'genbank')

1

In [183]:
# make a dictionary with the genes transferred by ratt
# rv_genes_in_erdman[locus_tag] = location

rv_genes_in_erdman = {}

for record in SeqIO.parse(fix_embl_id_line(ratt_file), 'embl'):
    for feature in record.features:
        # get only gene features
        if feature.type == 'gene':
            
            # get rv locus tag
            rv_locus_tag = feature.qualifiers['locus_tag'][0]

            # add location and strand
            rv_genes_in_erdman[rv_locus_tag] = {}
            rv_genes_in_erdman[rv_locus_tag]['start'] = feature.location.start
            rv_genes_in_erdman[rv_locus_tag]['end'] = feature.location.end
            rv_genes_in_erdman[rv_locus_tag]['strand'] = feature.location.strand   


ratt_features = []

for record in SeqIO.parse(fix_embl_id_line(ratt_file), 'embl'):
    for feature in record.features:
        # ignore these features:
        if feature.type not in ['source', 'repeat_region', 'gene', 'mobile_element']:
            
            # make a new feature with selected information

            # get location
            new_feature_start = feature.location.start
            new_feature_end = feature.location.end
            new_feature_strand = feature.location.strand

            if new_feature_strand == 1:
                new_feature_strand_symbol = '+'
            elif new_feature_strand == -1:
                new_feature_strand_symbol = '-'

            # make dictionary to store qualifiers of feature
            new_feature_qualifiers = feature.qualifiers

            # add if there's an Rv locus tag
            if 'locus_tag' in feature.qualifiers.keys():
                rv_id = feature.qualifiers['locus_tag'][0]
            else:
                rv_id = None
 
            # fix the CDS start
            
            # if there's a gene annotation with the same rv_id
            if rv_id in rv_genes_in_erdman.keys():
                # if the strand is +, update the "start" with what the rv_genes_in_erdman dictionary says
                if new_feature_strand == 1:
                    new_feature_start = rv_genes_in_erdman[rv_id]['start']
                elif new_feature_strand == -1:
                    # if the strand is -, update the "end"
                    new_feature_end = rv_genes_in_erdman[rv_id]['end']
                
            # make new feature
            new_feature = SeqFeature(FeatureLocation(new_feature_start, 
                                                     new_feature_end, 
                                                     strand=new_feature_strand),
                                     type=feature.type,
                                     qualifiers=new_feature_qualifiers)


            feature_line = ['ErdmanSF2024', feature.type, new_feature_start+1, 
                            new_feature_end, new_feature_strand_symbol, 
                            rv_id, 'ratt', new_feature]
            ratt_features.append(feature_line)

df_ratt = pd.DataFrame(ratt_features, columns=['chrom', 'type', 'start', 'end', 
                                               'strand', 'rv_equivalent', 'source', 'seqrecord'])

In [184]:
df_ratt.head()

,chrom,type,start,end,strand,rv_equivalent,source,seqrecord
0,ErdmanSF2024,CDS,1,1524,+,Rv0001,ratt,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
1,ErdmanSF2024,CDS,2052,3260,+,Rv0002,ratt,"(2051, 2052, 2053, 2054, 2055, 2056, 2057, 205..."
2,ErdmanSF2024,CDS,3280,4437,+,Rv0003,ratt,"(3279, 3280, 3281, 3282, 3283, 3284, 3285, 328..."
3,ErdmanSF2024,CDS,4434,4997,+,Rv0004,ratt,"(4433, 4434, 4435, 4436, 4437, 4438, 4439, 444..."
4,ErdmanSF2024,CDS,5240,7267,+,Rv0005,ratt,"(5239, 5240, 5241, 5242, 5243, 5244, 5245, 524..."


In [185]:
df_ratt['seqrecord'].apply(lambda x: x.type).value_counts()

seqrecord
CDS             3877
tRNA              45
misc_feature      33
ncRNA             20
rRNA               3
misc_RNA           2
Name: count, dtype: int64

## 2. Extract annotation features made _de novo_ with Bakta.

In [186]:
bakta_features = []

for record in SeqIO.parse(bakta_file, 'genbank'):
    for feature in record.features:
        # ignore these features:
        if feature.type not in ['repeat_region', 'gene', 'mobile_element', 'source']:
            
            # make a new feature with selected information

            # get location
            new_feature_start = feature.location.start
            new_feature_end = feature.location.end
            new_feature_strand = feature.location.strand

            if new_feature_strand == 1:
                new_feature_strand_symbol = '+'
            elif new_feature_strand == -1:
                new_feature_strand_symbol = '-'

            # make dictionary to store qualifiers of feature
            new_feature_qualifiers = feature.qualifiers
                
            # make new feature
            new_feature = SeqFeature(FeatureLocation(new_feature_start, 
                                                     new_feature_end, 
                                                     strand=new_feature_strand),
                                     type=feature.type,
                                     qualifiers=new_feature_qualifiers)


            feature_line = ['ErdmanSF2024', feature.type, new_feature_start+1, new_feature_end, new_feature_strand_symbol, None, 'bakta', new_feature]
            bakta_features.append(feature_line)

df_bakta = pd.DataFrame(bakta_features, 
                       columns=['chrom', 'type', 'start', 'end', 'strand', 
                                'rv_equivalent', 'source', 'seqrecord'])

In [187]:
!ls -1 ../../Results/Bakta_annotation/

ls: ../../Results/Bakta_annotation/: No such file or directory


In [188]:
df_bakta.head()

,chrom,type,start,end,strand,rv_equivalent,source,seqrecord
0,ErdmanSF2024,CDS,1,1524,+,None,bakta,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
1,ErdmanSF2024,rep_origin,1525,2051,+,None,bakta,"(1524, 1525, 1526, 1527, 1528, 1529, 1530, 153..."
2,ErdmanSF2024,CDS,2052,3260,+,None,bakta,"(2051, 2052, 2053, 2054, 2055, 2056, 2057, 205..."
3,ErdmanSF2024,CDS,3280,4437,+,None,bakta,"(3279, 3280, 3281, 3282, 3283, 3284, 3285, 328..."
4,ErdmanSF2024,CDS,4482,4997,+,None,bakta,"(4481, 4482, 4483, 4484, 4485, 4486, 4487, 448..."


In [189]:
df_bakta['seqrecord'].apply(lambda x: x.type).value_counts()

seqrecord
CDS           4070
tRNA            46
ncRNA           23
regulatory      13
rRNA             3
rep_origin       1
tmRNA            1
Name: count, dtype: int64

## 3. From the Bakta annotations, keep only those non-overlapping with RATT annotation

In [190]:
overlaps = bf.count_overlaps(df_bakta, df_ratt)
to_transfer_from_bakta = overlaps[overlaps['count'] == 0]

df_all_features = pd.concat([df_ratt, to_transfer_from_bakta]).sort_values('start')

In [191]:
df_all_features.head()

,chrom,type,start,end,strand,rv_equivalent,source,seqrecord,count
0,ErdmanSF2024,CDS,1,1524,+,Rv0001,ratt,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",NaN
1,ErdmanSF2024,rep_origin,1525,2051,+,None,bakta,"(1524, 1525, 1526, 1527, 1528, 1529, 1530, 153...",0.0
1,ErdmanSF2024,CDS,2052,3260,+,Rv0002,ratt,"(2051, 2052, 2053, 2054, 2055, 2056, 2057, 205...",NaN
2,ErdmanSF2024,CDS,3280,4437,+,Rv0003,ratt,"(3279, 3280, 3281, 3282, 3283, 3284, 3285, 328...",NaN
3,ErdmanSF2024,CDS,4434,4997,+,Rv0004,ratt,"(4433, 4434, 4435, 4436, 4437, 4438, 4439, 444...",NaN


In [192]:
df_all_features['type'].value_counts()

type
CDS             4025
tRNA              46
misc_feature      33
ncRNA             22
regulatory         6
rRNA               3
misc_RNA           2
rep_origin         1
Name: count, dtype: int64

## 4. Build genbank record
This describes the heuristics of what to transfer from the ratt and Bakta annotations

In [193]:
i = 1
new_features = []
gene_column = []
erd_locus_tag_column = []

for column, row in df_all_features.iterrows():
    seqrecord = row['seqrecord']
    qualifiers = {}
    
    if row['type']  == 'CDS':
        # locus_tag
        erdman_locus_tag = f'tmp_Erdman_{str(i).zfill(4)}'
        qualifiers['locus_tag'] = erdman_locus_tag
        i += 1

        # gene
        if 'gene' in seqrecord.qualifiers:
            qualifiers['gene'] = seqrecord.qualifiers['gene']
        elif row['source'] == 'ratt':
            if 'locus_tag' in seqrecord.qualifiers:
                rv_locus_tag = seqrecord.qualifiers['locus_tag'][0]
                qualifiers['gene'] = [f'E{rv_locus_tag}']
        else:
            qualifiers['gene'] = [erdman_locus_tag]

        # product
        if 'product' in seqrecord.qualifiers:
            qualifiers['product'] = seqrecord.qualifiers['product']

        # codon_start
        if 'codon_start' in seqrecord.qualifiers:
            qualifiers['codon_start'] = seqrecord.qualifiers['codon_start']
            
        # transl_table
        if 'transl_table' in seqrecord.qualifiers:
            qualifiers['transl_table'] = seqrecord.qualifiers['transl_table']
            
        # db_xref
        # if row['source'] == 'bakta':
        #     if 'db_xref' in seqrecord.qualifiers:
        #         qualifiers['db_xref'] = seqrecord.qualifiers['db_xref']

        # inference
        if row['source'] == 'ratt':
            qualifiers['inference'] = 'alignment:ratt:1.0.3:RefSeq:NC_000962.3'
        elif row['source'] == 'bakta':
            if 'inference' in seqrecord.qualifiers:
                qualifiers['inference'] = seqrecord.qualifiers['inference']

        # note
        if row['source'] == 'ratt':
            if 'locus_tag' in seqrecord.qualifiers:
                rv_locus_tag = seqrecord.qualifiers['locus_tag'][0]
                qualifiers['note'] = f'Simliar to {rv_locus_tag} of M. tuberculosis H37Rv'
        elif row['source'] == 'bakta':
            qualifiers['note'] = 'De novo annotation with bakta'

    elif row['type'] in ['tRNA', 'ncRNA', 'rRNA', 'misc_RNA']:
        # locus_tag
        erdman_locus_tag = f'tmp_Erdman_{str(i).zfill(4)}'
        qualifiers['locus_tag'] = erdman_locus_tag
        i += 1
        
        # gene
        if 'gene' in seqrecord.qualifiers:
            qualifiers['gene'] = seqrecord.qualifiers['gene']
        elif row['source'] == 'ratt':
            #print("")
            if 'locus_tag' in seqrecord.qualifiers:
                qualifiers['gene'] = None
            #if 'locus_tag' in seqrecord.qualifiers:
            #    rv_locus_tag = seqrecord.qualifiers['locus_tag'][0]
            #    qualifiers['gene'] = [f'E{rv_locus_tag}']
        else:
            print("")
            qualifiers['gene'] = [erdman_locus_tag]

        # product
        if 'product' in seqrecord.qualifiers:
            qualifiers['product'] = seqrecord.qualifiers['product']

        # db_xref
        # if row['source'] == 'bakta':
        #     if 'db_xref' in seqrecord.qualifiers:
        #         qualifiers['db_xref'] = seqrecord.qualifiers['db_xref']

        # inference
        if row['source'] == 'ratt':
            qualifiers['inference'] = 'alignment:ratt:1.0.3:RefSeq:NC_000962.3'
        elif row['source'] == 'bakta':
            if 'inference' in seqrecord.qualifiers:
                qualifiers['inference'] = seqrecord.qualifiers['inference']

        # note
        if row['source'] == 'ratt':
            if 'locus_tag' in seqrecord.qualifiers:
                rv_locus_tag = seqrecord.qualifiers['locus_tag'][0]
                qualifiers['note'] = f'Simliar to {rv_locus_tag} of M. tuberculosis H37Rv'
        elif row['source'] == 'bakta':
            qualifiers['note'] = 'De novo annotation with bakta'

        # special transfer only for ncRNA
        if row['type'] == 'ncRNA':
            qualifiers['ncRNA_class'] = seqrecord.qualifiers['ncRNA_class']

    elif row['type'] == 'misc_feature':
        # some are in reality pseudogenes
        if 'pseudogene' in seqrecord.qualifiers:
            seqrecord.type = 'CDS'
            
            # locus_tag
            erdman_locus_tag = f'tmp_Erdman_{str(i).zfill(4)}'
            qualifiers['locus_tag'] = erdman_locus_tag
            i += 1
            
            # gene
            if 'gene' in seqrecord.qualifiers:
                qualifiers['gene'] = seqrecord.qualifiers['gene']
            elif row['source'] == 'ratt':
                if 'locus_tag' in seqrecord.qualifiers:
                    rv_locus_tag = seqrecord.qualifiers['locus_tag'][0]
                    qualifiers['gene'] = [f'E{rv_locus_tag}']
            else:
                qualifiers['gene'] = [erdman_locus_tag]

            # product
            if 'note' in seqrecord.qualifiers:
                qualifiers['product'] =  seqrecord.qualifiers['note'][0]

            # note
            if row['source'] == 'ratt':
                if 'locus_tag' in seqrecord.qualifiers:
                    rv_locus_tag = seqrecord.qualifiers['locus_tag'][0]
                    qualifiers['note'] = f'Simliar to {rv_locus_tag} of M. tuberculosis H37Rv'
            elif row['source'] == 'bakta':
                qualifiers['note'] = 'De novo annotation with bakta'

            qualifiers['pseudo'] = None
            # qualifiers['pseudogene'] = 'unknown'
        else:
            seqrecord = None
        
    elif row['type'] == 'regulatory':
        # regulatory_class
        if 'regulatory_class' in seqrecord.qualifiers:
            if row['source'] == 'bakta':
                qualifiers['regulatory_class'] = seqrecord.qualifiers['regulatory_class']

        # db_xref
        # if row['source'] == 'bakta':
        #     if 'db_xref' in seqrecord.qualifiers:
        #         qualifiers['db_xref'] = seqrecord.qualifiers['db_xref']

        # inference
        if row['source'] == 'ratt':
            qualifiers['inference'] = 'alignment:ratt:1.0.3:RefSeq:NC_000962.3'
        elif row['source'] == 'bakta':
            if 'inference' in seqrecord.qualifiers:
                qualifiers['inference'] = seqrecord.qualifiers['inference']

        # function
        if row['source'] == 'bakta':
            if 'note' in seqrecord.qualifiers:
                qualifiers['function'] = seqrecord.qualifiers['note']

        # note
        if row['source'] == 'ratt':
            if 'locus_tag' in seqrecord.qualifiers:
                rv_locus_tag = seqrecord.qualifiers['locus_tag'][0]
                qualifiers['note'] = f'Simliar to {rv_locus_tag} of M. tuberculosis H37Rv'
        elif row['source'] == 'bakta':
            qualifiers['note'] = 'De novo annotation with bakta'

    elif row['type'] == 'rep_origin':
        # function
        if row['source'] == 'bakta':
            if 'note' in seqrecord.qualifiers:
                qualifiers['function'] = seqrecord.qualifiers['note']

        # note
        if row['source'] == 'bakta':
            qualifiers['note'] = 'De novo annotation with bakta'

    # make new feature
    if seqrecord:
        new_feature = SeqFeature(seqrecord.location,
                                 type=seqrecord.type,
                                 qualifiers=qualifiers)
    
        new_features.append(new_feature)




##### Manually adding barcodng plasmid features #####

# Here I will manually add the barcoding plasmid features which range from 2764911 to 2770126

# 1) general annotation of barcoding plasmid region
pjeb_barcoding_vector_Feat = SeqFeature(
                        location=FeatureLocation(2764911, 2770126, strand=+1),  # Specify the start and end positions
                        type="misc_feature",  # Specify the feature type
                        qualifiers={"label" : "pjeb_barcoding_vector",
                                    "note": "Inserted barcoding plasmid region"}  # Name the feature
)

new_features.append(pjeb_barcoding_vector_Feat)

# 2) attP_core_sequence

attP_core_sequence_Feat = SeqFeature(location=FeatureLocation(2764912, 2764955, strand=+1),  # Specify the start and end positions
                                type="misc_feature",  # Specify the feature type
                                qualifiers={"label" : "attP_core_sequence",
                                            "note": "barcoding plasmid feature - attP_core_sequence"}  # Name the feature
                            )

new_features.append(attP_core_sequence_Feat)


# 3) qTag_v27
qTag_v27_Feat = SeqFeature(location=FeatureLocation(2765549, 2765720, strand=+1),  # Specify the start and end positions
                                type="misc_feature",  # Specify the feature type
                                qualifiers={"label" : "qTag_v27",
                                            "note": "barcoding plasmid feature - qTag_v27"}  # Name the feature
                            )

new_features.append(qTag_v27_Feat)

# 4) OriE
OriE_Feat = SeqFeature(location=FeatureLocation(2765750, 2766543, strand=+1),  # Specify the start and end positions
                                type="misc_feature",  # Specify the feature type
                                qualifiers={"label" : "OriE",
                                            "note": "barcoding plasmid feature - OriE"}  # Name the feature
                            )

new_features.append(OriE_Feat)


# 5) barcode_random_7mer

barcode_random_7mer_Feat = SeqFeature(location=FeatureLocation(2765720, 2765727, strand=+1),  # Specify the start and end positions
                                type="misc_feature",  # Specify the feature type
                                qualifiers={"label" : "barcode_random_7mer",
                                            "note": "barcoding plasmid feature - barcode_random_7mer"}  # Name the feature
                            )

new_features.append(barcode_random_7mer_Feat)

#################################



new_record = list(SeqIO.parse(fix_embl_id_line(ratt_file), 'embl'))[0]
new_record.id = 'ErdmanSF2024'
new_record.name = 'ErdmanSF2024'

source_feature = new_record.features[0]

source_feature.qualifiers = {'strain': ['Erdman'],
                             'mol_type': ['genomic DNA'],
                             'db_xref': ['taxon:652616'],
                             'organism': ['Mycobacterium tuberculosis strain Erdman']}
new_features.insert(0, source_feature)

new_record.features = new_features

## 5. Manually fix CDS transferred with invalid ORF.
The file 'manual_fix.tsv' was made by manually inspecting cases of invalid orfs and by fetching the right coordinates.

In [194]:
df_fix = pd.read_csv('manual_fix.tsv',
                     sep='\t')


df_fix[['start', 'end']] = df_fix[['start', 'end']].fillna(0).astype(int)

fix_dict = {}
for column, row in df_fix.iterrows():
    gene = row['gene']
    fix_dict[gene] = {
        'position': (row['start'], row['end']),
        'action': row['action'].split(';'),
        'note': row['note'] if row['note'] != 'NaN' else None
    }


clean_features = []

# clean features according to what's specified in 'action'
for feature in new_record.features:
    if 'gene' in feature.qualifiers:
        gene = feature.qualifiers['gene'][0]
        if gene in fix_dict.keys():
            for task in fix_dict[gene]['action']:
                if task == 'update_position':
                    feature.location = FeatureLocation(fix_dict[gene]['position'][0]-1,
                                                       fix_dict[gene]['position'][1],
                                                       strand=feature.location.strand)
                if task == 'change_strand':
                    if feature.location.strand:
                        feature.location.strand = -1
                    else:
                        feature.location.strand = 1
                elif task == 'add_note':
                    feature.qualifiers['note'] += fix_dict[gene]['note']
                elif task == 'pseudo':
                    feature.qualifiers['pseudo'] = ''
                elif task == 'delete':
                    feature = None

    # Rerun all DELETE and position-update tasks using the locus tag as well
    if feature:
        if 'locus_tag' in feature.qualifiers:
            i_locus_tag = feature.qualifiers['locus_tag']
            if i_locus_tag in fix_dict.keys() and i_locus_tag != gene:
                for task in fix_dict[i_locus_tag]['action']:
                    if task == 'delete':
                        feature = None
                    if task == 'update_position':
                        feature.location = FeatureLocation(fix_dict[i_locus_tag]['position'][0]-1,
                                                           fix_dict[i_locus_tag]['position'][1],
                                                           strand=feature.location.strand)

    # get translation   
    if feature:
        if feature.type == 'CDS':
            if 'pseudo' not in feature.qualifiers:
                if feature.location.strand > 0:
                    subseq = new_record.seq[feature.location.start:feature.location.end]
                    aa = subseq.translate()
                else:
                    subseq = new_record.seq[feature.location.start:feature.location.end]
                    aa = subseq.reverse_complement().translate()
                feature.qualifiers['translation'] = Seq(aa)
        
        # clean_features.append(feature)



    #### Add a label for KanR gene and L5 Integrase

        if 'gene' in feature.qualifiers:
            gene_name = feature.qualifiers["gene"][0]
            if gene_name == "aph(3')-Ia":
                feature.qualifiers["label"] = ["aph_KanR"]
                feature.qualifiers["note"] = ['Barcoding plasmid: aph_KanR, kanamycin resistance gene']

            if gene_name == "tmp_Erdman_2579":
                feature.qualifiers["label"] = ["L5 integrase"]
                feature.qualifiers['gene'][0] = 'L5-gp33'
                feature.qualifiers["product"][0] = "site specific integrase"
                feature.qualifiers["note"] = ["Barcoding plasmid: site specific integrase (L5 gp33) (UniProt: P22884)"]
    
            if gene_name == "tmp_Erdman_2578":
                feature.qualifiers["label"] = ["L5 transcriptional repressor"]
                feature.qualifiers['gene'][0] = 'L5-gp71'
                feature.qualifiers["product"][0] = "L5 transcriptional repressor"
                feature.qualifiers["note"] = ["Barcoding plasmid: L5 transcriptional repressor (L5 gp71) (UniProt: Q05286)"]

        clean_features.append(feature)


# RESORT THE clean_features LIST

clean_features = sorted(clean_features, key=lambda feature: feature.location.start)

# add updated locus_tag
i = 1

for feature in clean_features:
    if 'locus_tag' in feature.qualifiers:
        erdman_locus_tag = f'ErdFL_{str(i).zfill(4)}'

        feature.qualifiers['locus_tag'] = erdman_locus_tag
        i += 1

In [195]:
len(clean_features)

4099

## 5.2 Add matching gene features for CDS and tRNA types

In [196]:
clean_features_WiGeneFeats = []

for feature in clean_features:

    # Retrieve qualifiers (if they exist)
    if feature.type in ["CDS", 'tRNA', 'ncRNA', 'rRNA', 'misc_RNA']:
        locus_tag = feature.qualifiers['locus_tag']
        gene = feature.qualifiers.get('gene', [''])[0]
        note = feature.qualifiers.get('gene', [''])[0]
        if gene == ['']: gene = ''
            
        # Create a new SeqFeature of type "gene"
        new_gene_feature = SeqFeature(
                location=feature.location,  # Reuse the same location
                type="gene",
                qualifiers={
                    'locus_tag': locus_tag,
                    'gene': [gene],
                }
            )

        if 'note' in feature.qualifiers:
            new_gene_feature.qualifiers["note"] = feature.qualifiers["note"]

        
        clean_features_WiGeneFeats.append(new_gene_feature)

    if 'gene' in feature.qualifiers:
        gene = feature.qualifiers['gene'][0]
        
        if 'tmp_Erdman' in gene:
            del feature.qualifiers['gene']
            del new_gene_feature.qualifiers['gene']
            gene = locus_tag # None

        if 'ERv' in gene:
            del feature.qualifiers['gene']
            del new_gene_feature.qualifiers['gene']

        if gene == "tnp":
            del feature.qualifiers['gene']
            del new_gene_feature.qualifiers['gene']

        if gene == "hypB":
            del feature.qualifiers['gene']
            del new_gene_feature.qualifiers['gene']


    
    clean_features_WiGeneFeats.append(feature)


## 6. Build feature table and rename `gene` to `locus_tag`

In [197]:
annotations = []

for feature in clean_features_WiGeneFeats:
    if 'locus_tag' in feature.qualifiers:
        locus_tag = feature.qualifiers['locus_tag']
    else:
        locus_tag = None
    if 'gene' in feature.qualifiers:
        gene = feature.qualifiers['gene'][0]
        
        # if 'tmp_Erdman' in gene:
        #     #gene = locus_tag
        #     #feature.qualifiers['gene'][0] = locus_tag
        #     del feature.qualifiers['gene']
        #     gene = locus_tag # None

        # if 'ERv' in gene:
        #     del feature.qualifiers['gene']
        # if gene == "tnp":
        #     del feature.qualifiers['gene']

    
    else:
        gene = locus_tag
        #gene = None
    
    record_type = feature.type
    start = feature.location.start
    end = feature.location.end
    if feature.location.strand > 0:
        strand = '+'
    else:
        strand = '-'
    
    if 'note' in feature.qualifiers:
        note = feature.qualifiers['note']
        if 'bakta' in note:
            source = 'bakta'
            rv_equivalent = None
            
        elif 'H37Rv' in note:
            source = 'ratt'
            rv_equivalent = note.split('Simliar to ')[1].split(' of M. tuberculosis H37Rv')[0]
            
    else:
        rv_equivalent = None
        source = None
        note = ""
    line = [locus_tag, gene, record_type, rv_equivalent, start, end, strand, source, note]
    annotations.append(line)

df_annotations = pd.DataFrame(annotations, columns = ['locus_tag', 'gene', 'type', 'rv_equivalent', 'start', 'end', 'strand', 'source', 'note'])

### Output curated features TSV and Genbank files

In [198]:
!mkdir ./CuratedAnnotationV2

mkdir: ./CuratedAnnotationV2: File exists


In [199]:
df_annotations.to_csv('./CuratedAnnotationV2/Erdman.curated.V2.tsv', sep='\t', index=None)

In [200]:
# save Genbank
new_record.features = clean_features_WiGeneFeats
SeqIO.write(new_record, './CuratedAnnotationV2/Erdman.curated.V2.gb', 'genbank')

1

In [201]:
df_annotations["type"].value_counts()

type
gene            4086
CDS             4014
tRNA              46
ncRNA             21
regulatory         6
misc_feature       5
rRNA               3
misc_RNA           2
source             1
rep_origin         1
Name: count, dtype: int64

# 5. Quality control

In [202]:
for feature in new_record.features:
    if feature.type == 'CDS':
        if 'pseudo' not in feature.qualifiers:
            aa = feature.qualifiers['translation']
            problem = ''
            
            if '*' in aa[:-1]:
                problem += 'Pseudo;'
            
            if aa[0] not in ['M', 'V', 'L']:
                problem += 'Start;'
            
            if aa[-1] != '*':
                problem += 'End;'
                
            if problem != '':
                print(problem)
                print(feature)

Start;
type: CDS
location: [550979:551153](-)
qualifiers:
    Key: codon_start, Value: ['1']
    Key: gene, Value: ['mazE1']
    Key: inference, Value: alignment:ratt:1.0.3:RefSeq:NC_000962.3
    Key: locus_tag, Value: ErdFL_0477
    Key: note, Value: Simliar to Rv0456B of M. tuberculosis H37Rv;ATT start inherited from H37Rv
    Key: product, Value: ['antitoxin MazE1']
    Key: transl_table, Value: ['11']
    Key: translation, Value: ITTYYYVLLSVTTWVGLRHEAKRELVYRGRRSIGRMPREWACRRSRRFAANGVDAAR*

Start;
type: CDS
location: [837211:837739](+)
qualifiers:
    Key: codon_start, Value: ['1']
    Key: inference, Value: alignment:ratt:1.0.3:RefSeq:NC_000962.3
    Key: locus_tag, Value: ErdFL_0778
    Key: note, Value: Simliar to Rv0742 of M. tuberculosis H37Rv;ATA start inherited from H37Rv
    Key: product, Value: ['hypothetical protein']
    Key: transl_table, Value: ['11']
    Key: translation, Value: ISFVIAAPEAIAAAATDLASIGSTIGAANAAAAANTTAVLAAGADQVSVAIAAAFGAHGQAYQALSAQAATFHIQFVQALTAGAGSYAAAEA

This are the three cases where the I start codon is inherited from H37Rv

## Peak at the summary TSV

In [203]:
type(feature.qualifiers)

dict

In [204]:
df_annotations.head(20)

,locus_tag,gene,type,rv_equivalent,start,end,strand,source,note
0,None,None,source,None,0,4416075,+,None,
1,ErdFL_0001,dnaA,gene,Rv0001,0,1524,+,ratt,Simliar to Rv0001 of M. tuberculosis H37Rv
2,ErdFL_0001,dnaA,CDS,Rv0001,0,1524,+,ratt,Simliar to Rv0001 of M. tuberculosis H37Rv
3,None,None,rep_origin,None,1524,2051,+,bakta,De novo annotation with bakta
4,ErdFL_0002,dnaN,gene,Rv0002,2051,3260,+,ratt,Simliar to Rv0002 of M. tuberculosis H37Rv
5,ErdFL_0002,dnaN,CDS,Rv0002,2051,3260,+,ratt,Simliar to Rv0002 of M. tuberculosis H37Rv
6,ErdFL_0003,recF,gene,Rv0003,3279,4437,+,ratt,Simliar to Rv0003 of M. tuberculosis H37Rv
7,ErdFL_0003,recF,CDS,Rv0003,3279,4437,+,ratt,Simliar to Rv0003 of M. tuberculosis H37Rv
8,ErdFL_0004,ErdFL_0004,gene,Rv0004,4433,4997,+,ratt,Simliar to Rv0004 of M. tuberculosis H37Rv
9,ErdFL_0004,ErdFL_0004,CDS,Rv0004,4433,4997,+,ratt,Simliar to Rv0004 of M. tuberculosis H37Rv


In [206]:
df_annotations.head(1000).tail(20)

,locus_tag,gene,type,rv_equivalent,start,end,strand,source,note
980,ErdFL_0489,fadB2,CDS,Rv0468,562533,563394,+,ratt,Simliar to Rv0468 of M. tuberculosis H37Rv
981,ErdFL_0490,umaA,gene,Rv0469,563526,564387,+,ratt,Simliar to Rv0469 of M. tuberculosis H37Rv
982,ErdFL_0490,umaA,CDS,Rv0469,563526,564387,+,ratt,Simliar to Rv0469 of M. tuberculosis H37Rv
983,ErdFL_0491,pcaA,gene,Rv0470c,564486,565350,-,ratt,Simliar to Rv0470c of M. tuberculosis H37Rv
984,ErdFL_0491,pcaA,CDS,Rv0470c,564486,565350,-,ratt,Simliar to Rv0470c of M. tuberculosis H37Rv
985,ErdFL_0492,ErdFL_0492,gene,Rv0470A,565492,565933,-,ratt,Simliar to Rv0470A of M. tuberculosis H37Rv
986,ErdFL_0492,ErdFL_0492,CDS,Rv0470A,565492,565933,-,ratt,Simliar to Rv0470A of M. tuberculosis H37Rv
987,ErdFL_0493,ErdFL_0493,gene,Rv0471c,565863,566352,-,ratt,Simliar to Rv0471c of M. tuberculosis H37Rv
988,ErdFL_0493,ErdFL_0493,CDS,Rv0471c,565863,566352,-,ratt,Simliar to Rv0471c of M. tuberculosis H37Rv
989,ErdFL_0494,ErdFL_0494,gene,Rv0472c,566361,567066,-,ratt,Simliar to Rv0472c of M. tuberculosis H37Rv
